<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-agents/blob/main/mcp-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCP Agent
[code](https://replit.com/@matt/OpenAI-Agents-SDK#main.py)
[video](https://www.youtube.com/watch?v=Ta5J_2KFBGM)
[replit](https://docs.replit.com/getting-started/intro-replit)
[trace](https://platform.openai.com/traces)

In [1]:
# Install

!pip install agents
!pip install openai-agents

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 25.6 MB/s eta 0:00:00
  Created wheel for agents: filename=agents-1.4.0-py3-none-any.whl size=62714 sha256=0ebfaf54040fa670f67ecd936398d37b0566fe49333fc2124f80ce336b04983b
  Stored in directory: /root/.cache/pip/wheels/e5/82/e5/2790dbbc1ad6037f1001bc436ea963e0877fff918dddc74fe2
Successfully built agents
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 18.6 MB/s eta 0:00:00


In [2]:
# Imports

import os
from datetime import datetime
from agents import Agent, Runner, function_tool, trace
import gradio as gr
from google.colab import userdata
from datetime import datetime
import httpx
from bs4 import BeautifulSoup
from typing_extensions import TypedDict, Any

In [3]:
# Fetch tokens for openai and pushover

openai_api_key = userdata.get("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and starts with {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

pushover_user = userdata.get("PUSHOVER_USER")
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

pushover_token = userdata.get("PUSHOVER_TOKEN")
if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

pushover_url = "https://api.pushover.net/1/messages.json"

OpenAI API Key exists and starts with sk-proj-
Pushover user found and starts with u
Pushover token found and starts with a


In [4]:
# The usual starting point

os.environ["OPENAI_API_KEY"] = openai_api_key

In [5]:
# Make an agent with name, instructions, model

async def main_test():
  agent = Agent(name="Jokester2",
            instructions="You are a joke teller",
            model="gpt-4o-mini")

  with trace("Telling a joke"):
    result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
    print(result.final_output)

await main_test()

Why did the autonomous AI agent break up with its partner?

Because it couldn’t handle the emotional bandwidth!


In [7]:
# Here is the modified code to run in Colab

class Location(TypedDict):
    lat: float
    long: float

@function_tool
async def get_time() -> str:
    """Fetch the current time"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@function_tool
async def fetch_url(url: str) -> str:
    """Fetch content from a URL"""
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        return response.text

@function_tool
async def fetch_weather(location: Location) -> str:
    """Fetch the weather for a given location."""
    return "sunny"

async def main():
    """Run the agent"""
    agent = Agent(
        name="Assistant",
        model="gpt-4o-mini",
        instructions="You are a helpful assistant",
        tools=[get_time, fetch_url]
    )

    result = await Runner.run(
        agent, """
            Please get the content of https://sullivanexcavatinginc.com/ and summarize them.
            Return the summary as well as the time you got the content.
            """
    )
    print(result.final_output)

await main()

It seems that the website's content could not be directly retrieved in a readable format; it appears to be a web application structure with no significant textual content available in the response.

However, I obtained the current time when accessing the content: **December 15, 2025, 13:48:02**.

If you need more specific information, please let me know!
